In [1]:
!pip install openpyxl

  Using cached https://files.pythonhosted.org/packages/f0/da/572cbc0bc582390480bbd7c4e93d14dc46079778ed915b505dc494b37c57/jdcal-1.4.1-py2.py3-none-any.whl


In [2]:
import os
import io
import pandas as pd
from rdflib import ConjunctiveGraph
from rdflib.plugins.sparql.results.csvresults import CSVResultSerializer

pd.set_option('display.max_colwidth', 200)

if not os.path.exists('outputs'):
    os.makedirs('outputs')
    
g = ConjunctiveGraph()
g.parse('data.trig', format='trig')

def query(q):
    serializer = CSVResultSerializer(g.query(q))
    output = io.BytesIO()
    serializer.serialize(output)
    return pd.read_csv(io.StringIO(output.getvalue().decode('utf-8')), encoding='utf-8')
    
def write(df, fn):
    df.to_excel('outputs/{}'.format(fn), encoding='utf-8')

In [3]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?date ?ri_acronym ?ri_website ?ri_domain ?rep_label WHERE {
    ?g dcterms:date ?date .
    GRAPH ?g { 
        ?ri a rm:ResearchInfrastructure .
        ?ri envri:acronym ?ri_acronym . 
        ?ri envri:website ?ri_website .
        ?ri envri:hasDomain ?ri_domain .
        ?ri envri:hasRepository ?rep .
        ?rep a envri:Repository .
        ?rep rdfs:label ?rep_label .
    }
    FILTER (?date > "2019-01-01"^^xsd:date)
    # FILTER (?ri_acronym = "ICOS")
}
""")

display(q)
write(q, 'output-1.xlsx')

,date,ri_acronym,ri_website,ri_domain,rep_label
0,2019-04-18,Euro-Argo,http://www.euro-argo.eu,n78,Euro-Argo Data
1,2019-04-30,LW (Marine),http://www.euro-argo.eu,n162,EUROBIS
2,2019-04-30,LW (Marine),http://www.euro-argo.eu,n162,Marine Data Archive
3,2019-04-01,ICOS,http://www.icos-cp.eu,n117,Carbon Portal
4,2019-05-23,SDN,http://www.seadatanet.org,n6,SeaDataNet Central Data Products
5,2019-05-23,SDN,http://www.seadatanet.org,n6,SeaDataNet Common DAta Index (CDI)


In [4]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?rep_label ?vocab_name ?vocab_iri WHERE {
    [] a rm:ResearchInfrastructure ;
       envri:hasRepository [
         a envri:Repository ;
         rdfs:label ?rep_label ;
         envri:hasVocabularies [
           envri:hasName ?vocab_name ;
           envri:hasVocabularyIri ?vocab_iri 
         ]
       ]
}
""")

display(q)
write(q, 'output-2.xlsx')

,rep_label,vocab_name,vocab_iri
0,Carbon Portal,Darwin Core,http://purl.org/dc/elements/1.1/
1,Carbon Portal,ICOS,http://meta.icos-cp.eu/ontologies/cpmeta/
2,Carbon Portal,PROV-O,http://www.w3.org/ns/prov
3,SeaDataNet Central Data Products,NERC vocabulary service,http://vocab.nerc.ac.uk/collection/V22/current/
4,SeaDataNet Central Data Products,EDMO,https://edmo.seadatanet.org
5,SeaDataNet Central Data Products,ISO 19115/19139,http://standards.iso.org/ittf/PubliclyAvailableStandards/ISO_19139_Schemas/resources/codelist/ML_gmxCodelists.xml
6,SeaDataNet Central Data Products,SeaDataNet CDI,https://cdi.seadatanet.org
7,EUROBIS,Marine species,http://www.marinespecies.org/
8,EUROBIS,Marine Regions,http://www.marineregions.org/
9,EUROBIS,WoRMS,http://www.marinespecies.org/


In [5]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>

SELECT ?p WHERE {
    ?p envri:relatesTo fairterms:A1.2 .
}
""")

display(q)
write(q, 'output-3.xlsx')

,p
0,http://envri.eu/ns/hasAuthenticationMethod
1,http://envri.eu/ns/usesAuthorisationTechnique
2,http://envri.eu/ns/contentAccessAuthorizationRequired
3,http://envri.eu/ns/maintainsOwnUserDatabase


In [6]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:I1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-4.xlsx')

,l,p,o,li
0,ICOS Carbon Portal metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/ISO_19115,NaN
1,LW (Marine) EUROBIS metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/ISO_19115,NaN
2,ICOS Carbon Portal metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/GeoDCAT,NaN
3,SDN SeaDataNet Common DAta Index (CDI) metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/ISO_19115_19139,NaN
4,SDN SeaDataNet Central Data Products metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/SeaDataNetCommunityProfile,NaN
5,Euro-Argo Euro-Argo Data metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/ArgoUserManual,NaN
6,SDN SeaDataNet Central Data Products metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/ISO_19115_19139,NaN
7,Euro-Argo Euro-Argo Data metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/NetCDFCFChecker,NaN
8,SDN SeaDataNet Common DAta Index (CDI) metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/SeaDataNetCommunityProfile,NaN
9,LW (Marine) EUROBIS metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/EML,NaN


In [7]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?rep_label ?schema_name WHERE {
    [] a rm:ResearchInfrastructure ;
       envri:hasRepository [
         a envri:Repository ;
         rdfs:label ?rep_label ;
         envri:hasMetadata [
           envri:hasSchema [
             envri:hasSchemaName ?schema_name
           ]
         ]
       ]
}
""")

display(q)
write(q, 'output-5.xlsx')

,rep_label,schema_name
0,Carbon Portal,http://envri.eu/ns/ISO_19115
1,Carbon Portal,http://envri.eu/ns/GeoDCAT
2,SeaDataNet Central Data Products,http://envri.eu/ns/ISO_19115_19139
3,SeaDataNet Central Data Products,http://envri.eu/ns/SeaDataNetCommunityProfile
4,EUROBIS,http://envri.eu/ns/ISO_19115
5,EUROBIS,http://envri.eu/ns/EML
6,SeaDataNet Common DAta Index (CDI),http://envri.eu/ns/ISO_19115_19139
7,SeaDataNet Common DAta Index (CDI),http://envri.eu/ns/SeaDataNetCommunityProfile
8,Euro-Argo Data,http://envri.eu/ns/ArgoUserManual
9,Euro-Argo Data,http://envri.eu/ns/NetCDFCFChecker


In [9]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:I1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-I1.xlsx')

,l,p,o,li
0,ICOS Carbon Portal metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/ISO_19115,NaN
1,LW (Marine) EUROBIS metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/ISO_19115,NaN
2,ICOS Carbon Portal metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/GeoDCAT,NaN
3,SDN SeaDataNet Common DAta Index (CDI) metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/ISO_19115_19139,NaN
4,SDN SeaDataNet Central Data Products metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/SeaDataNetCommunityProfile,NaN
5,Euro-Argo Euro-Argo Data metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/ArgoUserManual,NaN
6,SDN SeaDataNet Central Data Products metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/ISO_19115_19139,NaN
7,Euro-Argo Euro-Argo Data metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/NetCDFCFChecker,NaN
8,SDN SeaDataNet Common DAta Index (CDI) metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/SeaDataNetCommunityProfile,NaN
9,LW (Marine) EUROBIS metadata schema,http://envri.eu/ns/hasSchemaName,http://envri.eu/ns/EML,NaN


In [10]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:A1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-A1.xlsx')

,l,p,o,li
0,SDN SeaDataNet Common DAta Index (CDI) data processing,http://envri.eu/ns/otherAnalysisServicesOffered,http://envri.eu/ns/Planned,NaN
1,Euro-Argo Euro-Argo Data data processing,http://envri.eu/ns/otherAnalysisServicesOffered,n96,http://envri.eu/ns/Elasticsearch
2,Euro-Argo Euro-Argo Data data processing,http://envri.eu/ns/otherAnalysisServicesOffered,n96,http://envri.eu/ns/Pangeo
3,Euro-Argo Euro-Argo Data data processing,http://envri.eu/ns/otherAnalysisServicesOffered,n96,http://envri.eu/ns/Cassandra
4,Euro-Argo Euro-Argo Data data processing,http://envri.eu/ns/otherAnalysisServicesOffered,n96,http://envri.eu/ns/Spark
5,ICOS Carbon Portal data processing,http://envri.eu/ns/otherAnalysisServicesOffered,n138,http://envri.eu/ns/JupiterNotebooks
6,SDN SeaDataNet Central Data Products data processing,http://envri.eu/ns/otherAnalysisServicesOffered,http://envri.eu/ns/Planned,NaN
7,LW (Marine) EUROBIS data processing,http://envri.eu/ns/otherAnalysisServicesOffered,n184,http://envri.eu/ns/RPackages
8,LW (Marine) Marine Data Archive data processing,http://envri.eu/ns/otherAnalysisServicesOffered,http://envri.eu/ns/None,NaN
9,Euro-Argo repository,http://envri.eu/ns/hasPersistencyGuaranty,50 years,NaN


In [11]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:A2 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-A2.xlsx')

,l,p,o,li


In [12]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:A1.1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-A1_1.xlsx')

,l,p,o,li
0,Euro-Argo Euro-Argo Data access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/FTP,NaN
1,SDN SeaDataNet Central Data Products access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/CAS,NaN
2,SDN SeaDataNet Common DAta Index (CDI) access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/CAS,NaN
3,LW (Marine) EUROBIS access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/HTTPPOST,NaN
4,LW (Marine) Marine Data Archive access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/HTTPPOST,NaN
5,ICOS Carbon Portal access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/HTTPGET,NaN
6,Euro-Argo Euro-Argo Data access mechanism,http://envri.eu/ns/hasAccessProtocolUrl,https://doi.org/10.17882/42182,NaN
7,SDN SeaDataNet Central Data Products access mechanism,http://envri.eu/ns/hasAccessProtocolUrl,http://vocab.nerc.ac.uk/collection/L07/current/,NaN
8,SDN SeaDataNet Common DAta Index (CDI) access mechanism,http://envri.eu/ns/hasAccessProtocolUrl,http://vocab.nerc.ac.uk/collection/L07/current/,NaN
9,LW (Marine) EUROBIS access mechanism,http://envri.eu/ns/hasAccessProtocolUrl,https://doi.org/10.17487%2FRFC2616,NaN


In [13]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:A1.2 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-A1_2.xlsx')

,l,p,o,li
0,Euro-Argo Euro-Argo Data access mechanism,http://envri.eu/ns/hasAuthenticationMethod,http://envri.eu/ns/VOID,NaN
1,SDN SeaDataNet Central Data Products access mechanism,http://envri.eu/ns/hasAuthenticationMethod,Marine-ID,NaN
2,SDN SeaDataNet Common DAta Index (CDI) access mechanism,http://envri.eu/ns/hasAuthenticationMethod,Marine-ID,NaN
3,LW (Marine) EUROBIS access mechanism,http://envri.eu/ns/hasAuthenticationMethod,account manually,NaN
4,LW (Marine) Marine Data Archive access mechanism,http://envri.eu/ns/hasAuthenticationMethod,account manually,NaN
5,ICOS Carbon Portal access mechanism,http://envri.eu/ns/hasAuthenticationMethod,OAuth paired with eduGain,NaN
6,Euro-Argo Euro-Argo Data access mechanism,http://envri.eu/ns/usesAuthorisationTechnique,http://envri.eu/ns/VOID,NaN
7,SDN SeaDataNet Central Data Products access mechanism,http://envri.eu/ns/usesAuthorisationTechnique,http://envri.eu/ns/OAUTH,NaN
8,SDN SeaDataNet Common DAta Index (CDI) access mechanism,http://envri.eu/ns/usesAuthorisationTechnique,http://envri.eu/ns/OAUTH,NaN
9,LW (Marine) EUROBIS access mechanism,http://envri.eu/ns/usesAuthorisationTechnique,http://envri.eu/ns/SSQLService,NaN


In [14]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:I2 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-I2.xlsx')

,l,p,o,li
0,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasVocabularyIri,http://vocab.nerc.ac.uk/collection/V22/current/,NaN
1,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasVocabularyIri,https://cdi.seadatanet.org,NaN
2,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasVocabularyIri,https://edmo.seadatanet.org,NaN
3,SDN SeaDataNet Common DAta Index (CDI) vocabularies,http://envri.eu/ns/hasVocabularyIri,https://edmo.seadatanet.org,NaN
4,ICOS Carbon Portal vocabularies,http://envri.eu/ns/hasVocabularyIri,http://meta.icos-cp.eu/ontologies/cpmeta/,NaN
5,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasVocabularyIri,http://standards.iso.org/ittf/PubliclyAvailableStandards/ISO_19139_Schemas/resources/codelist/ML_gmxCodelists.xml,NaN
6,LW (Marine) Marine Data Archive vocabularies,http://envri.eu/ns/hasVocabularyIri,http://www.marinespecies.org/,NaN
7,SDN SeaDataNet Common DAta Index (CDI) vocabularies,http://envri.eu/ns/hasVocabularyIri,http://vocab.nerc.ac.uk/collection/V22/current/,NaN
8,ICOS Carbon Portal vocabularies,http://envri.eu/ns/hasVocabularyIri,http://purl.org/dc/elements/1.1/,NaN
9,SDN SeaDataNet Common DAta Index (CDI) vocabularies,http://envri.eu/ns/hasVocabularyIri,https://cdi.seadatanet.org,NaN


In [15]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:I3 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-I3.xlsx')

,l,p,o,li


In [16]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:R1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-R1.xlsx')

,l,p,o,li


In [17]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:R1.1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-R1_1.xlsx')

,l,p,o,li
0,Euro-Argo Euro-Argo Data access mechanism,http://envri.eu/ns/dataLicenseIri,https://creativecommons.org/licenses/by/4.0/,NaN
1,SDN SeaDataNet Central Data Products access mechanism,http://envri.eu/ns/dataLicenseIri,https://www.seadatanet.org/Data-Access/License/1.0,NaN
2,SDN SeaDataNet Common DAta Index (CDI) access mechanism,http://envri.eu/ns/dataLicenseIri,https://www.seadatanet.org/Data-Access/License/1.0,NaN
3,ICOS Carbon Portal access mechanism,http://envri.eu/ns/dataLicenseIri,https://creativecommons.org/licenses/by/4.0/,NaN
4,Euro-Argo Euro-Argo Data access mechanism,http://envri.eu/ns/usesDataLicenses,n89,http://envri.eu/ns/CC-BY4.0
5,SDN SeaDataNet Central Data Products access mechanism,http://envri.eu/ns/usesDataLicenses,n49,http://envri.eu/ns/LocalLicense
6,SDN SeaDataNet Common DAta Index (CDI) access mechanism,http://envri.eu/ns/usesDataLicenses,n17,http://envri.eu/ns/LocalLicense
7,LW (Marine) EUROBIS access mechanism,http://envri.eu/ns/usesDataLicenses,n170,http://envri.eu/ns/CCBY_CC0
8,LW (Marine) Marine Data Archive access mechanism,http://envri.eu/ns/usesDataLicenses,n212,http://envri.eu/ns/CCBY_CC0
9,ICOS Carbon Portal access mechanism,http://envri.eu/ns/usesDataLicenses,n127,http://envri.eu/ns/CC-BY4.0


In [18]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:R1.2 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-R1_2.xlsx')

,l,p,o,li


In [19]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:R1.3 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-R1_3.xlsx')

,l,p,o,li
0,SeaDataNet Common DAta Index (CDI) data management plans,http://envri.eu/ns/hasComplianceValidationService,False,NaN
1,Carbon Portal data management plans,http://envri.eu/ns/hasComplianceValidationService,False,NaN
2,Euro-Argo Data data management plans,http://envri.eu/ns/hasComplianceValidationService,True,NaN
3,SeaDataNet Central Data Products data management plans,http://envri.eu/ns/hasComplianceValidationService,False,NaN
4,Marine Data Archive data management plans,http://envri.eu/ns/hasComplianceValidationService,False,NaN
5,EUROBIS data management plans,http://envri.eu/ns/hasComplianceValidationService,False,NaN


In [20]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:F1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-F1.xlsx')

,l,p,o,li
0,ICOS Carbon Portal identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/DataCite,NaN
1,LW (Marine) Marine Data Archive identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/LocalService,NaN
2,SDN SeaDataNet Common DAta Index (CDI) identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/EUDAT,NaN
3,Euro-Argo Euro-Argo Data identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/SEANOE,NaN
4,LW (Marine) EUROBIS identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/DataCite,NaN
5,SDN SeaDataNet Central Data Products identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/DataCite,NaN
6,ICOS Carbon Portal identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/PDC,NaN
7,LW (Marine) Marine Data Archive identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/DataCite,NaN
8,LW (Marine) EUROBIS identifier,http://envri.eu/ns/usesProvider,http://envri.eu/ns/LocalService,NaN
9,ICOS Carbon Portal identifier,http://envri.eu/ns/usesIdentifierSystem,http://envri.eu/ns/DigitalObjectIdentifier,NaN


In [25]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:F2 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-F2.xlsx')

,l,p,o,li
0,ICOS Carbon Portal metadata,http://envri.eu/ns/hasMachineReadableDatasetMetadata,https://hdl.handle.net/11676/-ffoiHjX5NDN0Vq_fKuVmas0,NaN
1,LW (Marine) EUROBIS metadata,http://envri.eu/ns/hasMachineReadableDatasetMetadata,http://www.lifewatch.be/en/imis?module=dataset&dasid=1841&show=eml,NaN
2,SDN SeaDataNet Central Data Products metadata,http://envri.eu/ns/hasMachineReadableDatasetMetadata,http://seadatanet.maris2.nl/v_cdi_v3/print_xml.asp?n_code=2626292,NaN
3,LW (Marine) Marine Data Archive metadata,http://envri.eu/ns/hasMachineReadableDatasetMetadata,http://www.lifewatch.be/en/imis?module=dataset&dasid=1841&show=eml,NaN
4,SDN SeaDataNet Common DAta Index (CDI) metadata,http://envri.eu/ns/hasMachineReadableDatasetMetadata,https://doi.org/10.17882/42182,NaN
5,Euro-Argo Euro-Argo Data metadata,http://envri.eu/ns/hasMachineReadableDatasetMetadata,https://doi.org/10.17882/42182,NaN
6,LW (Marine) Marine Data Archive identifier,http://envri.eu/ns/includesMetadataSchema,http://envri.eu/ns/None,NaN
7,ICOS Carbon Portal identifier,http://envri.eu/ns/includesMetadataSchema,http://envri.eu/ns/None,NaN
8,LW (Marine) EUROBIS identifier,http://envri.eu/ns/includesMetadataSchema,http://envri.eu/ns/None,NaN
9,ICOS Carbon Portal metadata schema,http://envri.eu/ns/includesProvenanceFields,n144,http://envri.eu/ns/TextOnly


In [23]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:F3 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-F3.xlsx')

,l,p,o,li
0,Euro-Argo repository,http://envri.eu/ns/hasMetadata,n99,NaN
1,LW (Marine) repository,http://envri.eu/ns/hasMetadata,n217,NaN
2,ICOS repository,http://envri.eu/ns/hasMetadata,n141,NaN
3,SDN repository,http://envri.eu/ns/hasMetadata,n25,NaN
4,SDN repository,http://envri.eu/ns/hasMetadata,n60,NaN
5,LW (Marine) repository,http://envri.eu/ns/hasMetadata,n187,NaN
6,ICOS Carbon Portal metadata,http://envri.eu/ns/persistentIdentifiersAreIncluded,true,NaN
7,LW (Marine) EUROBIS metadata,http://envri.eu/ns/persistentIdentifiersAreIncluded,true,NaN
8,SDN SeaDataNet Central Data Products metadata,http://envri.eu/ns/persistentIdentifiersAreIncluded,true,NaN
9,LW (Marine) Marine Data Archive metadata,http://envri.eu/ns/persistentIdentifiersAreIncluded,true,NaN


In [24]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:F4 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-F4.xlsx')

,l,p,o,li
0,ICOS Carbon Portal metadata,http://envri.eu/ns/hasLocalSearchEngineUrl,https://meta.icos-cp.eu/sparqlclient/,NaN
1,LW (Marine) EUROBIS metadata,http://envri.eu/ns/hasLocalSearchEngineUrl,http://www.vliz.be/en/integrated-marine-information-system,NaN
2,SDN SeaDataNet Central Data Products metadata,http://envri.eu/ns/hasLocalSearchEngineUrl,https://cdi.seadatanet.org/search,NaN
3,LW (Marine) Marine Data Archive metadata,http://envri.eu/ns/hasLocalSearchEngineUrl,http://www.vliz.be/en/integrated-marine-information-system,NaN
4,SDN SeaDataNet Common DAta Index (CDI) metadata,http://envri.eu/ns/hasLocalSearchEngineUrl,https://www.seadatanet.org/Products#/search,NaN
5,Euro-Argo repository,http://envri.eu/ns/hasRepositoryUrl,http://doi.org/10.17882/42182,NaN
6,LW (Marine) repository,http://envri.eu/ns/hasRepositoryUrl,http://www.marinedataarchive.org/,NaN
7,ICOS repository,http://envri.eu/ns/hasRepositoryUrl,https://data.icos-cp.eu/portal,NaN
8,SDN repository,http://envri.eu/ns/hasRepositoryUrl,https://cdi.seadatanet.org/search,NaN
9,SDN repository,http://envri.eu/ns/hasRepositoryUrl,https://www.seadatanet.org/Products,NaN
